# Twitter Sentiment Analysis

In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
## Loading Data
df=pd.read_csv('Tweets.csv') # excel,sqllite
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df_new=df[['airline_sentiment','text']]

In [5]:
## Stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [6]:
stop_words=list(stop_words)

In [7]:
stop_words

['this',
 'hadn',
 'wouldn',
 'each',
 'itself',
 'wasn',
 'most',
 'which',
 'below',
 'no',
 'but',
 'its',
 'herself',
 'is',
 'for',
 'll',
 'her',
 'o',
 "you've",
 'those',
 'only',
 "needn't",
 "shan't",
 "mightn't",
 'how',
 'what',
 'off',
 's',
 'd',
 'under',
 'their',
 'haven',
 'both',
 'they',
 'when',
 'hers',
 'his',
 "doesn't",
 'was',
 'up',
 'he',
 "you'd",
 'them',
 'than',
 'to',
 'until',
 'had',
 'so',
 'am',
 "shouldn't",
 't',
 "isn't",
 'being',
 'where',
 'few',
 'weren',
 'i',
 'your',
 'has',
 'having',
 "she's",
 'it',
 'from',
 'hasn',
 'out',
 'did',
 "didn't",
 'are',
 'been',
 'there',
 'yourselves',
 'now',
 'own',
 'of',
 "it's",
 "hasn't",
 'very',
 'our',
 'a',
 "aren't",
 'or',
 'doesn',
 'by',
 'these',
 'through',
 'further',
 'ma',
 'yourself',
 'after',
 "couldn't",
 "you'll",
 'if',
 're',
 'ain',
 "wouldn't",
 'do',
 'about',
 'y',
 'same',
 'other',
 "haven't",
 'you',
 'should',
 "wasn't",
 'ours',
 'have',
 'ourselves',
 'won',
 'in',
 'o

In [8]:
import re
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stop_words]
    temp = " ".join(word for word in temp)
    return temp

In [9]:
df_new['airline_sentiment'].unique()
# x=df_new['text']
# y=df_new['airline_sentiment']

array(['neutral', 'positive', 'negative'], dtype=object)

In [10]:
df_new=df_new[df_new['airline_sentiment']!='neutral']

In [11]:
df_new['airline_sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [12]:
df_new.count()

airline_sentiment    11541
text                 11541
dtype: int64

In [13]:
df_new['airline_sentiment']=df_new['airline_sentiment'].map({'positive':1,'negative':0})

In [14]:
x=df_new['text']
y=df_new['airline_sentiment']

In [15]:
## Splitting train and Test Data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [16]:
x_train=x_train.apply(clean_tweet)

In [17]:
x_test=x_test.apply(clean_tweet)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vec=CountVectorizer()
x_train=vec.fit_transform(x_train)
x_test=vec.transform(x_test)

In [24]:
vec.get_feature_names_out()

array(['00', '000', '000419', ..., 'zones', 'zoom', 'zurich'],
      dtype=object)

In [21]:
## Classification
clf=XGBClassifier()
clf.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:07:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
y_pred=clf.predict(x_test)

In [24]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9042875703767865
